In [9]:
import nltk
import string

from collections import Counter

def get_tokens():
   with open('test-txt-search/1-s2.0-S8756328205000050-main.txt', 'r') as shakes:
    text = shakes.read()
    lowers = text.lower()
    #remove the punctuation using the character deletion step of translate
    translating = str.maketrans('', '', string.punctuation)
    no_punctuation = lowers.translate(translating)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

tokens = get_tokens()
count = Counter(tokens)
print(count.most_common(10))

[('of', 271), ('the', 200), ('and', 121), ('in', 120), ('a', 89), ('to', 72), ('were', 70), ('with', 69), ('was', 56), ('for', 49)]


In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')


tokens = get_tokens()
filtered = [w for w in tokens if not w in stopwords.words('english')]
count = Counter(filtered)
print(count.most_common(100))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rditljej/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('infection', 45), ('bone', 45), ('application', 44), ('systemic', 43), ('gentamicin', 41), ('animals', 34), ('j', 32), ('surgery', 31), ('implants', 31), ('groups', 31), ('antibiotic', 28), ('local', 25), ('prophylaxis', 24), ('1', 23), ('body', 19), ('osteomyelitis', 18), ('aureus', 18), ('al', 18), ('group', 18), ('2', 18), ('10', 17), ('implant', 17), ('study', 16), ('germany', 15), ('coating', 15), ('treated', 15), ('bacterial', 15), ('et', 15), ('b', 14), ('r', 14), ('g', 14), ('wires', 13), ('uncoated', 13), ('cultures', 13), ('sterile', 13), ('score', 13), ('pdlla', 12), ('kirschner', 12), ('coated', 12), ('tissue', 12), ('lucke', 12), ('iv', 12), ('2005', 11), ('titanium', 11), ('weight', 11), ('36', 11), ('blood', 11), ('h', 11), ('ii', 11), ('iii', 11), ('orthopedic', 10), ('antibiotics', 10), ('determined', 10), ('formation', 10), ('new', 10), ('f', 10), ('v', 10), ('c', 9), ('24', 9), ('30', 9), ('could', 9), ('770–778', 9), ('culture', 9), ('levels', 9), ('joint', 9), ('

In [11]:


from nltk.stem.porter import *

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)
count = Counter(stemmed)
print(count.most_common(100))



[('infect', 59), ('implant', 53), ('system', 50), ('group', 49), ('bone', 47), ('applic', 45), ('anim', 42), ('gentamicin', 41), ('antibiot', 39), ('j', 32), ('surgeri', 31), ('local', 28), ('coat', 27), ('cultur', 25), ('prophylaxi', 24), ('1', 23), ('bodi', 20), ('score', 19), ('osteomyel', 18), ('aureu', 18), ('use', 18), ('al', 18), ('2', 18), ('10', 17), ('studi', 17), ('wire', 16), ('determin', 16), ('treat', 16), ('germani', 15), ('bacteri', 15), ('et', 15), ('b', 14), ('r', 14), ('g', 14), ('prevent', 14), ('effect', 14), ('uncoat', 13), ('steril', 13), ('rat', 12), ('orthoped', 12), ('pdlla', 12), ('kirschner', 12), ('tissu', 12), ('luck', 12), ('iv', 12), ('2005', 11), ('titanium', 11), ('follow', 11), ('weight', 11), ('36', 11), ('blood', 11), ('radiograph', 11), ('h', 11), ('ii', 11), ('iii', 11), ('model', 10), ('administr', 10), ('evalu', 10), ('compar', 10), ('histolog', 10), ('develop', 10), ('format', 10), ('new', 10), ('level', 10), ('observ', 10), ('joint', 10), ('f'

In [3]:
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

path = 'test-txt-search/'
token_dict = {}
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

for subdir, dirs, files in os.walk(path):
    for file in files:
        file_path = subdir + os.path.sep + file
        shakes = open(file_path, 'r', encoding="utf-8")
        text = shakes.read()
        lowers = text.lower()
        translating = str.maketrans('', '', string.punctuation)
        no_punctuation = lowers.translate(translating)
        token_dict[file] = no_punctuation
        
#this can take some time
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

In [4]:
tfidf

TfidfVectorizer(stop_words='english',
                tokenizer=<function tokenize at 0x1a367cae0>)

In [5]:
tfs

<29x20348 sparse matrix of type '<class 'numpy.float64'>'
	with 42880 stored elements in Compressed Sparse Row format>

In [6]:
tfs.toarray()

array([[0.00472504, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01056117, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02124357, 0.        , 0.        , ..., 0.        , 0.        ,
        0.00337378],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00747705, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00457502, 0.        , 0.        , ..., 0.        , 0.        ,
        0.01198854]])

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

text_files = 'test-txt-search/'

my_files = [open(text_files + f, encoding="utf-8").read() for f in os.listdir(text_files)]



vectorizer = CountVectorizer()
features = vectorizer.fit_transform(my_files).todense()
print(type(features))
features = np.ravel(features)
print(type(features))
features = features.reshape(-1, 1)
print(vectorizer.vocabulary)


for f in features:
    print(euclidean_distances(features[0], f) )






<class 'numpy.matrix'>
<class 'numpy.ndarray'>
None


ValueError: Expected 2D array, got 1D array instead:
array=[5.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os

text_files = 'test-txt-search/'

documents = [open(text_files + f).read() for f in os.listdir(text_files)]

tfidf2 = TfidfVectorizer().fit_transform(documents)
pairwise_similarity = tfidf2 * tfidf2.T
pairwise_similarity.toarray()



array([[1.        , 0.45796436, 0.38054368, 0.53336731, 0.35163531,
        0.50066485, 0.48392548, 0.38773982, 0.39807556, 0.46231201,
        0.4491752 , 0.47036461, 0.45656694, 0.52675202, 0.40217485,
        0.51862055, 0.45039195, 0.4867068 , 0.43578735, 0.47871998,
        0.46710477, 0.40240653, 0.39713527, 0.38403346, 0.39071106,
        0.39906123, 0.37946456, 0.3574044 , 0.44429348],
       [0.45796436, 1.        , 0.4223986 , 0.64511004, 0.44592726,
        0.55438792, 0.50651414, 0.48098081, 0.44537114, 0.48729242,
        0.55811798, 0.522876  , 0.52816706, 0.54279441, 0.51558293,
        0.53740693, 0.54928577, 0.53797214, 0.49095306, 0.46340212,
        0.51920259, 0.52263881, 0.48641214, 0.48813257, 0.4263046 ,
        0.49950029, 0.45758097, 0.43958291, 0.50476807],
       [0.38054368, 0.4223986 , 1.        , 0.51650739, 0.36882628,
        0.47101217, 0.48816395, 0.35354589, 0.3657146 , 0.47733872,
        0.42349398, 0.45574767, 0.44452676, 0.37977091, 0.35540749,
  

In [21]:
import numpy as np

arr = pairwise_similarity.toarray()
np.fill_diagonal(arr, np.nan)




with open('1-s2.0-S8756328205000050-main.txt', 'r') as shakes:
    text = shakes.read()

input_doc = text
input_idx = documents.index(input_doc)
print(input_idx)

ValueError: 'Systemic versus local application of gentamicin in prophylaxis of\nimplant-related osteomyelitis in a rat model\nM. Luckea,T, B. Wildemanna, S. Sadonia, C. Surkea, R. Schillerb,\nA. Stembergerc, M. Raschked, N.P. Haasa, G. Schmidmaiera\naCenter for Musculoskeletal Surgery, Charite ´—University Medicine, Campus Virchow, D-13353 Berlin, Germany\nbDepartment of Microbiology and Hygiene Charite ´—University Medicine Berlin, Germany\ncInstitute of Experimental Surgery—Technical University of Munich, Germany\ndDepartment of Trauma, Hand and Reconstructive Surgery—University of Mu ¨nster, Germany\nReceived 14 April 2004; revised 10 December 2004; accepted 10 January 2005\nAvailable online 24 March 2005\nAbstract\nAdministration of perioperative antibiotic prophylaxis is a routine procedure in orthopedic surgery. Besides systemic prophylaxis, only\nfew techniques are established for local application of antibiotics to reduce infection related to orthopedic implants.\nThe aim of th

In [25]:
import os
import nltk
import nltk.corpus

In [28]:
from nltk.corpus import brown
nltk.download('brown')
brown.words()

[nltk_data] Downloading package brown to /Users/rditljej/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [30]:
nltk.download('gutenberg')
nltk.corpus.gutenberg.fileids()

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/rditljej/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [32]:
hamlet = nltk.corpus.gutenberg.words('shakespeare-hamlet.txt')
hamlet

['[', 'The', 'Tragedie', 'of', 'Hamlet', 'by', ...]

In [33]:
for word in hamlet[:500]:
    print(word, sep=' ', end=' ')

[ The Tragedie of Hamlet by William Shakespeare 1599 ] Actus Primus . Scoena Prima . Enter Barnardo and Francisco two Centinels . Barnardo . Who ' s there ? Fran . Nay answer me : Stand & vnfold your selfe Bar . Long liue the King Fran . Barnardo ? Bar . He Fran . You come most carefully vpon your houre Bar . ' Tis now strook twelue , get thee to bed Francisco Fran . For this releefe much thankes : ' Tis bitter cold , And I am sicke at heart Barn . Haue you had quiet Guard ? Fran . Not a Mouse stirring Barn . Well , goodnight . If you do meet Horatio and Marcellus , the Riuals of my Watch , bid them make hast . Enter Horatio and Marcellus . Fran . I thinke I heare them . Stand : who ' s there ? Hor . Friends to this ground Mar . And Leige - men to the Dane Fran . Giue you good night Mar . O farwel honest Soldier , who hath relieu ' d you ? Fra . Barnardo ha ' s my place : giue you goodnight . Exit Fran . Mar . Holla Barnardo Bar . Say , what is Horatio there ? Hor . A peece of him Bar 

In [34]:
with open('1-s2.0-S8756328205000050-main.txt', encoding='utf-8') as f:
    AI = f.read()

In [35]:
type(AI)

str

In [36]:
from nltk.tokenize import word_tokenize

In [37]:
AI_tokens = word_tokenize(AI)
AI_tokens

['Systemic',
 'versus',
 'local',
 'application',
 'of',
 'gentamicin',
 'in',
 'prophylaxis',
 'of',
 'implant-related',
 'osteomyelitis',
 'in',
 'a',
 'rat',
 'model',
 'M.',
 'Luckea',
 ',',
 'T',
 ',',
 'B.',
 'Wildemanna',
 ',',
 'S.',
 'Sadonia',
 ',',
 'C.',
 'Surkea',
 ',',
 'R.',
 'Schillerb',
 ',',
 'A.',
 'Stembergerc',
 ',',
 'M.',
 'Raschked',
 ',',
 'N.P',
 '.',
 'Haasa',
 ',',
 'G.',
 'Schmidmaiera',
 'aCenter',
 'for',
 'Musculoskeletal',
 'Surgery',
 ',',
 'Charite',
 '´—University',
 'Medicine',
 ',',
 'Campus',
 'Virchow',
 ',',
 'D-13353',
 'Berlin',
 ',',
 'Germany',
 'bDepartment',
 'of',
 'Microbiology',
 'and',
 'Hygiene',
 'Charite',
 '´—University',
 'Medicine',
 'Berlin',
 ',',
 'Germany',
 'cInstitute',
 'of',
 'Experimental',
 'Surgery—Technical',
 'University',
 'of',
 'Munich',
 ',',
 'Germany',
 'dDepartment',
 'of',
 'Trauma',
 ',',
 'Hand',
 'and',
 'Reconstructive',
 'Surgery—University',
 'of',
 'Mu',
 '¨nster',
 ',',
 'Germany',
 'Received',
 '14',

In [38]:
len(AI_tokens)

7283

In [39]:
from nltk.probability import FreqDist
fdist = FreqDist()

In [41]:
for word in AI_tokens:
    fdist[word.lower()] += 1
fdist

FreqDist({',': 404, '.': 397, 'of': 271, 'the': 201, ')': 156, '(': 155, 'and': 122, 'in': 119, '[': 95, ']': 95, ...})

In [42]:
fdist['coating']

16

In [43]:
len(fdist)

2013

In [45]:
fdist_top10 = fdist.most_common(40)
fdist_top10

[(',', 404),
 ('.', 397),
 ('of', 271),
 ('the', 201),
 (')', 156),
 ('(', 155),
 ('and', 122),
 ('in', 119),
 ('[', 95),
 (']', 95),
 ('a', 88),
 (';', 78),
 ('to', 73),
 ('were', 70),
 ('with', 69),
 ('was', 56),
 ('for', 49),
 ('infection', 46),
 ('bone', 46),
 ('application', 45),
 ('systemic', 43),
 ('gentamicin', 43),
 ('from', 34),
 ('animals', 34),
 ('groups', 33),
 ('surgery', 31),
 ('implants', 31),
 ('j', 30),
 ('antibiotic', 28),
 ('be', 27),
 ('local', 26),
 ('prophylaxis', 24),
 (':', 24),
 ('1', 23),
 ('by', 22),
 ('is', 19),
 ('body', 19),
 ('on', 19),
 ('osteomyelitis', 18),
 ('aureus', 18)]

In [46]:
from nltk.tokenize import blankline_tokenize
AI_blank = blankline_tokenize(AI)
len(AI_blank)

1

In [47]:
AI_blank[0]

'Systemic versus local application of gentamicin in prophylaxis of\nimplant-related osteomyelitis in a rat model\nM. Luckea,T, B. Wildemanna, S. Sadonia, C. Surkea, R. Schillerb,\nA. Stembergerc, M. Raschked, N.P. Haasa, G. Schmidmaiera\naCenter for Musculoskeletal Surgery, Charite ´—University Medicine, Campus Virchow, D-13353 Berlin, Germany\nbDepartment of Microbiology and Hygiene Charite ´—University Medicine Berlin, Germany\ncInstitute of Experimental Surgery—Technical University of Munich, Germany\ndDepartment of Trauma, Hand and Reconstructive Surgery—University of Mu ¨nster, Germany\nReceived 14 April 2004; revised 10 December 2004; accepted 10 January 2005\nAvailable online 24 March 2005\nAbstract\nAdministration of perioperative antibiotic prophylaxis is a routine procedure in orthopedic surgery. Besides systemic prophylaxis, only\nfew techniques are established for local application of antibiotics to reduce infection related to orthopedic implants.\nThe aim of this study was

In [48]:
from nltk.util import bigrams, trigrams, ngrams

In [52]:
quote_bigrams = list(nltk.trigrams(AI_tokens))
quote_bigrams

[('Systemic', 'versus', 'local'),
 ('versus', 'local', 'application'),
 ('local', 'application', 'of'),
 ('application', 'of', 'gentamicin'),
 ('of', 'gentamicin', 'in'),
 ('gentamicin', 'in', 'prophylaxis'),
 ('in', 'prophylaxis', 'of'),
 ('prophylaxis', 'of', 'implant-related'),
 ('of', 'implant-related', 'osteomyelitis'),
 ('implant-related', 'osteomyelitis', 'in'),
 ('osteomyelitis', 'in', 'a'),
 ('in', 'a', 'rat'),
 ('a', 'rat', 'model'),
 ('rat', 'model', 'M.'),
 ('model', 'M.', 'Luckea'),
 ('M.', 'Luckea', ','),
 ('Luckea', ',', 'T'),
 (',', 'T', ','),
 ('T', ',', 'B.'),
 (',', 'B.', 'Wildemanna'),
 ('B.', 'Wildemanna', ','),
 ('Wildemanna', ',', 'S.'),
 (',', 'S.', 'Sadonia'),
 ('S.', 'Sadonia', ','),
 ('Sadonia', ',', 'C.'),
 (',', 'C.', 'Surkea'),
 ('C.', 'Surkea', ','),
 ('Surkea', ',', 'R.'),
 (',', 'R.', 'Schillerb'),
 ('R.', 'Schillerb', ','),
 ('Schillerb', ',', 'A.'),
 (',', 'A.', 'Stembergerc'),
 ('A.', 'Stembergerc', ','),
 ('Stembergerc', ',', 'M.'),
 (',', 'M.', 'Ra

In [54]:
quotes_ngrams = list(nltk.ngrams(AI_tokens, 5))
quotes_ngrams

[('Systemic', 'versus', 'local', 'application', 'of'),
 ('versus', 'local', 'application', 'of', 'gentamicin'),
 ('local', 'application', 'of', 'gentamicin', 'in'),
 ('application', 'of', 'gentamicin', 'in', 'prophylaxis'),
 ('of', 'gentamicin', 'in', 'prophylaxis', 'of'),
 ('gentamicin', 'in', 'prophylaxis', 'of', 'implant-related'),
 ('in', 'prophylaxis', 'of', 'implant-related', 'osteomyelitis'),
 ('prophylaxis', 'of', 'implant-related', 'osteomyelitis', 'in'),
 ('of', 'implant-related', 'osteomyelitis', 'in', 'a'),
 ('implant-related', 'osteomyelitis', 'in', 'a', 'rat'),
 ('osteomyelitis', 'in', 'a', 'rat', 'model'),
 ('in', 'a', 'rat', 'model', 'M.'),
 ('a', 'rat', 'model', 'M.', 'Luckea'),
 ('rat', 'model', 'M.', 'Luckea', ','),
 ('model', 'M.', 'Luckea', ',', 'T'),
 ('M.', 'Luckea', ',', 'T', ','),
 ('Luckea', ',', 'T', ',', 'B.'),
 (',', 'T', ',', 'B.', 'Wildemanna'),
 ('T', ',', 'B.', 'Wildemanna', ','),
 (',', 'B.', 'Wildemanna', ',', 'S.'),
 ('B.', 'Wildemanna', ',', 'S.', '

In [55]:
from nltk.stem import PorterStemmer
pst = PorterStemmer()

In [56]:
pst.stem("having")

'have'

In [58]:
words_to_stem=['give', 'giving', 'giver', 'gave', 'given']
for words in words_to_stem:
    print(words + ":" + pst.stem(words))

give:give
giving:give
giver:giver
gave:gave
given:given


In [57]:
from nltk.stem import LancasterStemmer
lst = LancasterStemmer()
words_to_stem=['give', 'giving', 'giver', 'gave', 'given']
for words in words_to_stem:
    print(words + ":" + lst.stem(words))

give:giv
giving:giv
giver:giv
gave:gav
given:giv


In [59]:
from nltk.stem import SnowballStemmer
sbst = SnowballStemmer('english')

In [60]:
for words in words_to_stem:
    print(words + ":" + sbst.stem(words))

give:give
giving:give
giver:giver
gave:gave
given:given


In [61]:
from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer
word_lem = WordNetLemmatizer()

In [62]:
word_lem.lemmatize('corpora')

'corpus'

In [65]:
for words in words_to_stem:
    print(words + ":" + word_lem.lemmatize(words))

give:give
giving:giving
giver:giver
gave:gave
given:given


In [67]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [70]:
len(stopwords.words('english'))

179

In [71]:
fdist_top10

[(',', 404),
 ('.', 397),
 ('of', 271),
 ('the', 201),
 (')', 156),
 ('(', 155),
 ('and', 122),
 ('in', 119),
 ('[', 95),
 (']', 95),
 ('a', 88),
 (';', 78),
 ('to', 73),
 ('were', 70),
 ('with', 69),
 ('was', 56),
 ('for', 49),
 ('infection', 46),
 ('bone', 46),
 ('application', 45),
 ('systemic', 43),
 ('gentamicin', 43),
 ('from', 34),
 ('animals', 34),
 ('groups', 33),
 ('surgery', 31),
 ('implants', 31),
 ('j', 30),
 ('antibiotic', 28),
 ('be', 27),
 ('local', 26),
 ('prophylaxis', 24),
 (':', 24),
 ('1', 23),
 ('by', 22),
 ('is', 19),
 ('body', 19),
 ('on', 19),
 ('osteomyelitis', 18),
 ('aureus', 18)]

In [72]:
import re
punctuation=re.compile(r'[-.?!,:;()|0-9]')

In [73]:
post_punctuation = []
for words in AI_tokens:
    word = punctuation.sub("",words)
    if len(word) > 0:
        post_punctuation.append(word)

In [74]:
post_punctuation

['Systemic',
 'versus',
 'local',
 'application',
 'of',
 'gentamicin',
 'in',
 'prophylaxis',
 'of',
 'implantrelated',
 'osteomyelitis',
 'in',
 'a',
 'rat',
 'model',
 'M',
 'Luckea',
 'T',
 'B',
 'Wildemanna',
 'S',
 'Sadonia',
 'C',
 'Surkea',
 'R',
 'Schillerb',
 'A',
 'Stembergerc',
 'M',
 'Raschked',
 'NP',
 'Haasa',
 'G',
 'Schmidmaiera',
 'aCenter',
 'for',
 'Musculoskeletal',
 'Surgery',
 'Charite',
 '´—University',
 'Medicine',
 'Campus',
 'Virchow',
 'D',
 'Berlin',
 'Germany',
 'bDepartment',
 'of',
 'Microbiology',
 'and',
 'Hygiene',
 'Charite',
 '´—University',
 'Medicine',
 'Berlin',
 'Germany',
 'cInstitute',
 'of',
 'Experimental',
 'Surgery—Technical',
 'University',
 'of',
 'Munich',
 'Germany',
 'dDepartment',
 'of',
 'Trauma',
 'Hand',
 'and',
 'Reconstructive',
 'Surgery—University',
 'of',
 'Mu',
 '¨nster',
 'Germany',
 'Received',
 'April',
 'revised',
 'December',
 'accepted',
 'January',
 'Available',
 'online',
 'March',
 'Abstract',
 'Administration',
 'o

In [75]:
sent = "Timothy is a natural when it comes to drawing"
sent_tokens = word_tokenize(sent)

In [79]:
nltk.download('averaged_perceptron_tagger')
for token in AI_tokens:
    print(nltk.pos_tag([token]))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rditljej/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('Systemic', 'JJ')]
[('versus', 'NN')]
[('local', 'JJ')]
[('application', 'NN')]
[('of', 'IN')]
[('gentamicin', 'NN')]
[('in', 'IN')]
[('prophylaxis', 'NN')]
[('of', 'IN')]
[('implant-related', 'JJ')]
[('osteomyelitis', 'NN')]
[('in', 'IN')]
[('a', 'DT')]
[('rat', 'NN')]
[('model', 'NN')]
[('M.', 'NNP')]
[('Luckea', 'NN')]
[(',', ',')]
[('T', 'NN')]
[(',', ',')]
[('B.', 'NNP')]
[('Wildemanna', 'NNP')]
[(',', ',')]
[('S.', 'NNP')]
[('Sadonia', 'NN')]
[(',', ',')]
[('C.', 'NNP')]
[('Surkea', 'NN')]
[(',', ',')]
[('R.', 'NNP')]
[('Schillerb', 'NN')]
[(',', ',')]
[('A.', 'NN')]
[('Stembergerc', 'NN')]
[(',', ',')]
[('M.', 'NNP')]
[('Raschked', 'VBN')]
[(',', ',')]
[('N.P', 'NN')]
[('.', '.')]
[('Haasa', 'NN')]
[(',', ',')]
[('G.', 'NN')]
[('Schmidmaiera', 'NN')]
[('aCenter', 'NN')]
[('for', 'IN')]
[('Musculoskeletal', 'NN')]
[('Surgery', 'NN')]
[(',', ',')]
[('Charite', 'NN')]
[('´—University', 'NN')]
[('Medicine', 'NN')]
[(',', ',')]
[('Campus', 'NN')]
[('Virchow', 'NN')]
[(',', ',')]
[(

In [80]:
from nltk import ne_chunk

In [81]:
NE_sent = "The US President stays in the WHITE HOUSE"

In [82]:
NE_tokens = word_tokenize(NE_sent)
NE_tags = nltk.pos_tag(NE_tokens)
AI_tags = nltk.pos_tag(AI_tokens)

In [86]:
nltk.download('words')
nltk.download('maxent_ne_chunker')
NE_NER = ne_chunk(NE_tags)
AI_NER = ne_chunk(AI_tags)
print(AI_NER)

[nltk_data] Downloading package words to /Users/rditljej/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/rditljej/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


(S
  (GPE Systemic/NNP)
  versus/NN
  local/JJ
  application/NN
  of/IN
  gentamicin/NN
  in/IN
  prophylaxis/NN
  of/IN
  implant-related/JJ
  osteomyelitis/NN
  in/IN
  a/DT
  rat/NN
  model/NN
  M./NNP
  Luckea/NNP
  ,/,
  (GPE T/NNP)
  ,/,
  B./NNP
  Wildemanna/NNP
  ,/,
  S./NNP
  (PERSON Sadonia/NNP)
  ,/,
  C./NNP
  Surkea/NNP
  ,/,
  R./NNP
  Schillerb/NNP
  ,/,
  A./NNP
  Stembergerc/NNP
  ,/,
  M./NNP
  Raschked/NNP
  ,/,
  N.P/NNP
  ./.
  (PERSON Haasa/NNP)
  ,/,
  G./NNP
  Schmidmaiera/NNP
  aCenter/NN
  for/IN
  (ORGANIZATION Musculoskeletal/NNP Surgery/NNP)
  ,/,
  (PERSON Charite/NNP)
  ´—University/NNP
  Medicine/NNP
  ,/,
  (PERSON Campus/NNP Virchow/NNP)
  ,/,
  D-13353/NNP
  (PERSON Berlin/NNP)
  ,/,
  (GPE Germany/NNP)
  bDepartment/NN
  of/IN
  (GPE Microbiology/NNP)
  and/CC
  (PERSON Hygiene/NNP Charite/NNP)
  ´—University/NNP
  (PERSON Medicine/NNP Berlin/NNP)
  ,/,
  (GPE Germany/NNP)
  (ORGANIZATION cInstitute/NN of/IN Experimental/NNP)
  Surgery—Technical/NNP